# How does the taxonomic distribution of contigs differ by soil layer (mineral vs organic) in Colorado?

This notebook uses the `nmdc_api_utilities` package (as of March 2025) to explore how the taxononomic distribution of contigs differ by the mineral and organic soil layers in Colorado. It involves using `nmdc_api_utilites` objects to make NMDC API requests to reach the scaffold lineage TSV data objects in order to analyze the taxanomic distribution. Iterating through the TSV files includes 350+ API calls to get the necessary taxonomic counts and is time consuming. 

In [1]:
%%capture
## First need to install locally the nmdc_api_utilities
%pip install nmdc_api_utilities;
## And also install python-dotenv for development use
%pip install python-dotenv

## Set up environment variables
**You can disregard this section of code unless you have interest in testing this on the development API.**

Using pythons python-dotenv package, load the environment variables from the system. This chunk of code is used in the Github CI/CD pipelines to test our development API. The variable can be used when creating `nmdc_api_utilities` objects. If you do not have the environment variable `ENV` set in a .env file, this code with default to the variable "prod" - which tells nmdc_api_utilities to use the production API URL.

In [2]:
# set up environment variables
from dotenv import load_dotenv
import os
load_dotenv()
# load the environment variable ENV. If it does not exist, default to "prod"
ENV = os.environ.get("ENV", "prod")

In [3]:
import requests
import pandas as pd
from io import StringIO
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import nmdc_api_utilities

## 1. Get all biosamples where `soil_horizon` exists and the `geo_loc_name` has "Colorado" in the name 

The first step in answering how the taxonomic distribution of contigs differ by soil layer is to get a list of all the biosamples that have metadata for `soil_horizon` and a string matching "Colorado, Rocky Moutains" for the `geo_loc_name`. 
Using the Python package '`nmdc_api_utilities`', we can use the `get_record_by_filter` function to do this. We first need create a BiosampleSearch object to search across the "biosample_set" collections. More information regarding the nmdc_api_utilities package can be found [here](https://microbiomedata.github.io/nmdc_api_utilities/). We then create a mongo-like filter of `{"soil_horizon":{"$exists": true}, "geo_loc_name.has_raw_value": {"$regex": "Colorado"}}`, a maximum page size of 100, and specifying that we want three fields returned `id`, `soil_horizon`, and `geo_loc_name`. Note that `id` is returned no matter what. Since we will be joining the results of multiple API requests with a field of `id` for different collections, we can change the name of the `id` key to be more explicit - calling it `biosample_id` instead. Finally, we convert the biosample results to a dataframe called `biosample_df`. Note that about 517 biosamples are returned.

In [4]:
from nmdc_api_utilities.biosample_search import BiosampleSearch
from nmdc_api_utilities.data_processing import DataProcessing
# Create a BiosampleSearch object
bs_client = BiosampleSearch(env=ENV)
# create a DataProcessing object
dp_client = DataProcessing()
# define the filter
filter = '{"soil_horizon":{"$exists": true}, "geo_loc_name.has_raw_value": {"$regex": "Colorado"}}'
# get the results
bs_results = bs_client.get_record_by_filter(filter=filter, fields="id,soil_horizon,geo_loc_name", max_page_size=100, all_pages=True)
# clarify names
for biosample in bs_results:
    biosample["biosample_id"] = biosample.pop("id")

# convert to df
biosample_df = dp_client.convert_to_df(bs_results)

# Adjust geo_loc_name to not be a dictionary
biosample_df["geo_loc_name"] = biosample_df["geo_loc_name"].apply(lambda x: x.get("has_raw_value"))
biosample_df

,soil_horizon,geo_loc_name,biosample_id
0,M horizon,"USA: Colorado, Central Plains Experimental Range",nmdc:bsm-11-00m15h97
1,M horizon,"USA: Colorado, Central Plains Experimental Range",nmdc:bsm-11-06ta8e31
2,O horizon,"USA: Colorado, Rocky Mountains",nmdc:bsm-11-06tgpb52
3,M horizon,"USA: Colorado, Central Plains Experimental Range",nmdc:bsm-11-0asn5d63
4,M horizon,"USA: Colorado, North Sterling",nmdc:bsm-11-0djp2e45
...,...,...,...
513,M horizon,"USA: Colorado, Central Plains Experimental Range",nmdc:bsm-11-zhrzwh12
514,M horizon,"USA: Colorado, Niwot Ridge",nmdc:bsm-11-zhzner35
515,O horizon,"USA: Colorado, Niwot Ridge",nmdc:bsm-11-zjsrkd21
516,O horizon,"USA: Colorado, Niwot Ridge",nmdc:bsm-11-zk6h3328


## 2. Get all DataObjects that are related to the biosamples found in step 1. 

We'll do this using the `get_linked_instances_and_associate_ids` function from the `nmdc_api_utilities` package.

In [5]:
# Now get all DataObjects linked to these biosamples
biosample_dataobject_dictionary = bs_client.get_linked_instances_and_associate_ids(
    ids=biosample_df["biosample_id"].tolist(),          # list of biosample ids
    types =["nmdc:DataObject"]                          # specify we want DataObjects only
    )

# The result is a dictionary with biosample ids as keys and list of linked DataObjects as values
print(f"Retrieved DataObjects for {len(biosample_dataobject_dictionary)} biosamples.")

Retrieved DataObjects for 506 biosamples.


## 3. Get records for all DataObjects found in step 2.

To do this, we'll collect all the DataObject ids found in step 2 and use the `get_batch_records` function from the `nmdc_api_utilities` package. This function allows us to get records in batches, which is useful when dealing with a large number of ids. We'll specify that we want to retrieve the `id`, `data_object_type`, and `url` fields for each DataObject. The results will be stored in a dataframe called `dataobject_df`.

In [6]:
# Gather all DataObject ids into a single list (from the dictionary of lists), then get their records
dojs = [item for sublist in biosample_dataobject_dictionary.values() for item in sublist]

# Now get records for all DataObjects records and convert to dataframe
from nmdc_api_utilities.data_object_search import DataObjectSearch
do_client = DataObjectSearch()
do_results = do_client.get_batch_records(
    id_list=dojs,
    search_field="id",
    fields="id,data_object_type,url"
)
data_object_df = dp_client.convert_to_df(do_results)
# Rename id column to data_object_id for clarity
data_object_df = data_object_df.rename(columns={"id": "data_object_id"})

# Finally, filter these DataObjects to only include those with `data_object_type` of "Scaffold Lineage tsv"
filtered_data_object_df = data_object_df[data_object_df["data_object_type"] == "Scaffold Lineage tsv"]
filtered_data_object_df.head(10)

,data_object_id,data_object_type,url
57,nmdc:dobj-11-jbv50k17,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...
77,nmdc:dobj-11-rtrt0b26,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...
183,nmdc:dobj-11-t1k10324,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...
236,nmdc:dobj-11-cjkzhg87,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...
250,nmdc:dobj-11-gz9tyw27,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...
257,nmdc:dobj-11-jjeye060,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...
269,nmdc:dobj-11-nq8g1d37,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...
451,nmdc:dobj-11-e9d98533,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...
475,nmdc:dobj-11-s7dphe48,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...
550,nmdc:dobj-11-g87j5y46,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...


## 4. Associate DataObjects back to Biosamples

We'll use the `id` field from the `biosample_df` dataframe (which corresponds to ids of Biosample records) and the `id` field from the `dataobject_df` dataframe to associate DataObjects back to their respective Biosamples. This will allow us to link the taxonomic data in the DataObjects to the soil horizon information in the Biosamples.  The biosample_dataobject_dictionary created in step 2 will be used to map the relationships.

In [7]:
# First define a function to get biosample_id from data_object_id
def get_biosample_id(data_object_id, biosample_dataobject_dict):
    for biosample_id, data_object_list in biosample_dataobject_dict.items():
        if data_object_id in data_object_list:
            return biosample_id
    return None

# Apply the function to add biosample_id column
filtered_data_object_df["biosample_id"] = filtered_data_object_df["data_object_id"].apply(lambda x: get_biosample_id(x, biosample_dataobject_dictionary))

# Merge with biosample_df to get soil_horizon and geo_loc_name
merged_df = pd.merge(filtered_data_object_df, biosample_df, on="biosample_id", how="left")
merged_df.head(10)

/var/folders/dk/y9lfqwk971n02nvqf8d6c74h0000gn/T/ipykernel_24113/1847364105.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,data_object_id,data_object_type,url,biosample_id,soil_horizon,geo_loc_name
0,nmdc:dobj-11-jbv50k17,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,nmdc:bsm-11-1tbfgc11,O horizon,"USA: Colorado, Rocky Mountains"
1,nmdc:dobj-11-rtrt0b26,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,nmdc:bsm-11-kgch0q77,M horizon,"USA: Colorado, Rocky Mountains"
2,nmdc:dobj-11-t1k10324,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,nmdc:bsm-11-8w2k4265,M horizon,"USA: Colorado, Niwot Ridge"
3,nmdc:dobj-11-cjkzhg87,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,nmdc:bsm-11-bx1gz045,O horizon,"USA: Colorado, Niwot Ridge"
4,nmdc:dobj-11-gz9tyw27,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,nmdc:bsm-11-8e4hjr18,M horizon,"USA: Colorado, Central Plains Experimental Range"
5,nmdc:dobj-11-jjeye060,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,nmdc:bsm-11-3dmqhk09,M horizon,"USA: Colorado, Niwot Ridge"
6,nmdc:dobj-11-nq8g1d37,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,nmdc:bsm-11-4cf75j18,M horizon,"USA: Colorado, North Sterling"
7,nmdc:dobj-11-e9d98533,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,nmdc:bsm-11-47c1fp43,M horizon,"USA: Colorado, Central Plains Experimental Range"
8,nmdc:dobj-11-s7dphe48,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,nmdc:bsm-11-kdjqae05,O horizon,"USA: Colorado, Rocky Mountains"
9,nmdc:dobj-11-g87j5y46,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,nmdc:bsm-11-dwmydq07,M horizon,"USA: Colorado, Niwot Ridge"


## 5. Dereplicate the DataFrame based on data_object_id to ensure uniqueness

Prepare a final DataFrame that contains unique entries for each DataObject, ensuring that there are no duplicate records. This step is crucial for accurate analysis and reporting, as it prevents any potential bias or errors that could arise from counting the same DataObject multiple times which can result if several biosamples are pooled together in a single metagenome assembly and share the same DataObject.

In [8]:
# Prepare a final DataFrame that contains unique entries for each DataObject, ensuring that there are no duplicate records. This step is crucial for accurate analysis and reporting, as it prevents any potential bias or errors that could arise from counting the same DataObject multiple times which can result if several biosamples are pooled together in a single metagenome assembly and share the same DataObject.
final_df = merged_df[["data_object_id", "data_object_type", "url","soil_horizon", "geo_loc_name"]].drop_duplicates()
final_df.head(10)


,data_object_id,data_object_type,url,soil_horizon,geo_loc_name
0,nmdc:dobj-11-jbv50k17,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,O horizon,"USA: Colorado, Rocky Mountains"
1,nmdc:dobj-11-rtrt0b26,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,M horizon,"USA: Colorado, Rocky Mountains"
2,nmdc:dobj-11-t1k10324,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,M horizon,"USA: Colorado, Niwot Ridge"
3,nmdc:dobj-11-cjkzhg87,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,O horizon,"USA: Colorado, Niwot Ridge"
4,nmdc:dobj-11-gz9tyw27,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,M horizon,"USA: Colorado, Central Plains Experimental Range"
5,nmdc:dobj-11-jjeye060,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,M horizon,"USA: Colorado, Niwot Ridge"
6,nmdc:dobj-11-nq8g1d37,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,M horizon,"USA: Colorado, North Sterling"
7,nmdc:dobj-11-e9d98533,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,M horizon,"USA: Colorado, Central Plains Experimental Range"
8,nmdc:dobj-11-s7dphe48,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,O horizon,"USA: Colorado, Rocky Mountains"
9,nmdc:dobj-11-g87j5y46,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,M horizon,"USA: Colorado, Niwot Ridge"


## Show how many results have M horizon vs. O horizon

The `soil_horizon` column can be counted using the `value_counts()` functionality. There are many more M horizon samples than O horizon.

In [9]:
# Show unique soil horizons:
soil_horizons = final_df['soil_horizon'].value_counts()
print(soil_horizons)

soil_horizon
M horizon    165
O horizon     46
Name: count, dtype: int64


## Randomly select a subset of these datasets for which to pull information

In [10]:
# randomly select 15 data sets in each horizon
n = 15

#list the different types
list_type=soil_horizons.index.tolist()

#for each type, randomly horizon n data sets and save them into list
random_subset=[]
for type in list_type:
    #each data object ID and horizon type
    sample_type=final_df[['data_object_id','soil_horizon']].drop_duplicates()
    #filter to current horizon type
    sample_type=sample_type[sample_type['soil_horizon']==type]
    #randomly horizon n data object IDs in current horizon type
    sample_type=sample_type.sample(n=n, random_state=2)
    #save
    random_subset.append(sample_type)

#resave list as dataframe
random_subset=pd.concat(random_subset).reset_index(drop=True)

#remerge rest of the data for the sampled data sets
final_df=random_subset.merge(final_df,on=['data_object_id','soil_horizon'],how="left")

final_df

,data_object_id,soil_horizon,data_object_type,url,geo_loc_name
0,nmdc:dobj-11-h15bzm96,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, Central Plains Experimental Range"
1,nmdc:dobj-11-rc505c71,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, Niwot Ridge"
2,nmdc:dobj-11-xg738272,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,"USA: Colorado, North Sterling"
3,nmdc:dobj-11-byrh0203,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, Central Plains Experimental Range"
4,nmdc:dobj-11-k1vrrb83,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, North Sterling"
5,nmdc:dobj-11-jjeye060,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:dgns...,"USA: Colorado, Niwot Ridge"
6,nmdc:dobj-11-g87j5y46,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, Niwot Ridge"
7,nmdc:dobj-11-xt9amn82,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, Niwot Ridge"
8,nmdc:dobj-11-qc7nhb84,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, North Sterling"
9,nmdc:dobj-11-rkzz2g95,M horizon,Scaffold Lineage tsv,https://data.microbiomedata.org/data/nmdc:ompr...,"USA: Colorado, North Sterling"


## Example of what the TSV contig taxa file looks like

A snippet of the TSV file we need to iterate over to get the taxa abundance for the contigs is shown below. The third column is the initial count for the taxa, where each row is `1.0`. However, there are duplicate rows of taxa, meaning there are actually more than `1.0` for several taxa (though they appear as duplicate rows with `1.0` as the count`). We will take this into consideration when we calculate the relative abundance for each taxa.

In [11]:
tsv_ex_url = final_df.at[0, "url"]

response = requests.get(tsv_ex_url)
tsv_data = StringIO(response.text)

tsv_ex_df = pd.read_csv(tsv_data, delimiter="\t")
tsv_data.close()

# Give columns names
tsv_ex_df.columns = ["contig_id", "taxa", "initial_count"]

# sort by taxa
tsv_sorted = tsv_ex_df.sort_values(by="taxa")

# print first 10 rows
tsv_sorted[:10]

,contig_id,taxa,initial_count
65,nmdc:wfmgas-11-xsq6q988.1_scf_10067_c1,Archaea;Candidatus Thermoplasmatota;Thermoplas...,1.0
17055,nmdc:wfmgas-11-xsq6q988.1_scf_965_c1,Archaea;Candidatus Thermoplasmatota;Thermoplas...,1.0
12608,nmdc:wfmgas-11-xsq6q988.1_scf_509_c1,Archaea;Candidatus Thermoplasmatota;Thermoplas...,1.0
1848,nmdc:wfmgas-11-xsq6q988.1_scf_11970_c1,Archaea;Candidatus Thermoplasmatota;Thermoplas...,1.0
13195,nmdc:wfmgas-11-xsq6q988.1_scf_5693_c1,Archaea;Candidatus Thermoplasmatota;Thermoplas...,1.0
16452,nmdc:wfmgas-11-xsq6q988.1_scf_901_c1,Archaea;Candidatus Thermoplasmatota;Thermoplas...,1.0
8071,nmdc:wfmgas-11-xsq6q988.1_scf_19045_c1,Archaea;Euryarchaeota;Archaeoglobi;Archaeoglob...,1.0
8211,nmdc:wfmgas-11-xsq6q988.1_scf_19214_c1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,1.0
15731,nmdc:wfmgas-11-xsq6q988.1_scf_8261_c1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,1.0
1995,nmdc:wfmgas-11-xsq6q988.1_scf_12124_c1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,1.0


## Iterate throught the TSVs to get the contig taxa information

Using the Python `requests` library and the `StringIO` library, the TSV urls can be iterated over gathering the taxa information. The TSVs are converted into dataframes where they are manipulated to suit the data structure needed. The columns are given names and the taxa column is split into a proper list (instead of a string of items separated by a semicolon ;). The third element from the list of taxa is retrieved to get only the phylum level information of the taxa. A grouping function is performed on the `taxa` column and the Pandas `size()` functionality is used to calculate the count for how many times each taxa occurs, which is then used to calculate the relative abundance of each taxa for each biosample. After iterating through all of the TSVs, two final taxa dfs are created by concatenating the list of data frames (`o_df` and `m_df`). 

Any errors in requesting the TSV urls are collected as a dictionary, so we can either try to query them again, or look into why they were not able to be collected. 

Note this takes several hours to complete.

In [12]:
o_horizon = []
m_horizon = []
errors = []

iteration_counter = 0


for index, row in final_df.iterrows():
    
    iteration_counter += 1

    # print an update for every 50 iterations
    if iteration_counter % 50 == 0:
        print(f"Processed {iteration_counter} rows")

    url = row["url"]
    horizon = row["soil_horizon"]
    dataobj = row["data_object_id"]
    geo_loc = row["geo_loc_name"]
    data_object_id = row["data_object_id"]

    try:
        response = requests.get(url)
        tsv_data = StringIO(response.text)
    
        tsv_df = pd.read_csv(tsv_data, delimiter="\t")
        tsv_data.close()
    
        # Give columns names
        tsv_df.columns = ["contig_id", "taxa", "initial_count"]
    
        # split taxa column into a list where a semicolon (;) is the delimeter
        tsv_df["taxa"] = tsv_df["taxa"].str.split(";")

        # Get only the third element of the list of taxa (the phylum), add "Unknown" it it does not include phylum level, and add
        # "Unkown" if the taxa value is empty.
        tsv_df["taxa"] = tsv_df["taxa"].apply(lambda x: str(x[2]) if isinstance(x, list) and len(x) >= 3 
                                              else str(" ".join(x) + " Unknown") if isinstance(x, list) else "Unknown")


        # Get relative abundance for the tsv_df
        tsv_df = tsv_df.groupby("taxa").size().reset_index(name="count")
        total_count = tsv_df["count"].sum()
        tsv_df["relative_abundance"] = (tsv_df["count"] / total_count) * 100

        # Add geo location to data frame
        tsv_df["geo_loc_name"] = geo_loc

        # Add biosample id to data frame
        tsv_df["data_object_id"] = dataobj
        tsv_df["tsv_url"] = url

        # append tsv_df to list depending on the soil horizon type
        if horizon == "O horizon":
            o_horizon.append(tsv_df)
        else:
            m_horizon.append(tsv_df)

    except Exception as e:
        print(f"An error occurred: {e}")
        errors.append({
            "data_ob_id": dataobj,
            "url": url,
            "horizon": horizon,
            "geo_loc_name": geo_loc
                                    })
        continue

# concatenate list of dfs
o_df = pd.concat(o_horizon)
m_df = pd.concat(m_horizon)

m_df

,taxa,count,relative_abundance,geo_loc_name,data_object_id,tsv_url
0,Acidimicrobiia,341,1.960672,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-h15bzm96,https://data.microbiomedata.org/data/nmdc:ompr...
1,Acidithiobacillia,3,0.017249,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-h15bzm96,https://data.microbiomedata.org/data/nmdc:ompr...
2,Actinomycetes,11492,66.076357,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-h15bzm96,https://data.microbiomedata.org/data/nmdc:ompr...
3,Agaricomycetes,2,0.011500,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-h15bzm96,https://data.microbiomedata.org/data/nmdc:ompr...
4,Alphaproteobacteria,563,3.237121,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-h15bzm96,https://data.microbiomedata.org/data/nmdc:ompr...
...,...,...,...,...,...,...
206,unclassified Zoopagomycota,1,0.000840,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-7f86q517,https://data.microbiomedata.org/data/nmdc:dgns...
207,unclassified candidate division NC10,47,0.039484,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-7f86q517,https://data.microbiomedata.org/data/nmdc:dgns...
208,unclassified candidate division Zixibacteria,3,0.002520,"USA: Colorado, Central Plains Experimental Range",nmdc:dobj-11-7f86q517,https://data.microbiomedata.org/data/nmdc:dgns...
0,Actinomycetes,2,66.666667,"USA: Colorado, North Sterling",nmdc:dobj-11-vc747p06,https://data.microbiomedata.org/data/nmdc:ompr...


## Look into any errors that occurred from the TSV requests

Any TSVs that could not be requested were added to an `errors` dictionary. 

In [13]:
print(errors)

[]


## Define a function to calculate abundance

A function is defined that takes an input of a dataframe and calculates the average relative abundance of each taxa.

In [14]:
def taxa_abundance(df):

    df = df.drop_duplicates(subset=['data_object_id', 'taxa'])

    # pivot the table to find all combos of biosample and taxa - set NAs to 0 for relative abundance
    wide_df = df.pivot(index = "data_object_id", columns = "taxa", values = "relative_abundance")
    wide_df = wide_df.fillna(0)
    wide_df.reset_index(inplace=True)
    
    # convert wide_df back with relative_abundances set to 0 for samples that were missing taxa
    melted_df = pd.melt(wide_df, id_vars = "data_object_id", var_name = "taxa", value_name = "relative_abundance")

    # calculate abundance and add column to data frame
    final_df = melted_df.groupby("taxa")["relative_abundance"].mean().reset_index(name="avg_relative_abundance")

    return final_df

## Calculate the abundance of the O and M horizon data frames

Using the function defined above, the `counts_m` and `counts_o` data frames returned from iterating over the TSV files are used as input into the function, where the average relative abundance calculations are returned as data frames. We then concatenate the two data frames together, creating a new column for `soil_horizon`, where the value is either `O` or `M`, depending on which data frame it originally came from. 

In [15]:
# caculate abundance for each soil horizon type and get top 25 taxa, grouping the rest
m_final = taxa_abundance(m_df)
o_final = taxa_abundance(o_df)

# combine data frames
o_final["soil_horizon"] = "O"
m_final["soil_horizon"] = "M"
abundance_df = pd.concat([o_final, m_final])

abundance_df

,taxa,avg_relative_abundance,soil_horizon
0,Acidimicrobiia,0.409258,O
1,Acidithiobacillia,0.028993,O
2,Aconoidasida,0.003277,O
3,Actinomycetes,22.838596,O
4,Actinopteri,0.001862,O
...,...,...,...
282,unclassified Zoopagomycota,0.000111,M
283,unclassified candidate division NC10,0.072352,M
284,unclassified candidate division Zixibacteria,0.011305,M
285,"unclassified dsDNA viruses, no RNA stage",0.000005,M


## Plot the taxa abundance of M vs. O horizon soil samples

Using the `plotly` library, the percent abundance of the taxa is plotted as a bar chart - each bar representing the soil horizon and the colors representing the taxa.

In [16]:
# Plot the taxa abundance of each soil type
fig = px.bar(abundance_df, x="soil_horizon", y="avg_relative_abundance", color="taxa", 
             title = "% Abundance of phylum-level taxa in M and O horizon soil samples in Colorado", 
             labels = {"soil_horizon": "Soil Horizon", "avg_relative_abundance": "% Abundance"})
    
fig.update_layout(height=600)
fig.show()          

## Write a function to calculate the abundance per location

This is a function to use with the `m_df` and `o_df` outputs from the TSV iteration to calculate the % abundance for each `geo_loc_name`. It also groups the taxa after the top 5 for each loaction into "other".

In [17]:
def loc_abund(df):

    df = df.drop_duplicates(subset=['data_object_id', 'taxa'])

    # pivot the table to find all combos of biosample and taxa - set NAs to 0 for relative abundance
    wide_df = df.pivot(index = "data_object_id", columns = "taxa", values = "relative_abundance")
    wide_df = wide_df.fillna(0)
    wide_df.reset_index(inplace=True)

    # Add geo_loc_name column to wide_df
    wide_df = pd.merge(wide_df, df[['data_object_id', 'geo_loc_name']], on='data_object_id', how='left')
    
    # convert wide_df back with relative_abundances set to 0 for samples that were missing taxa
    melted_df = pd.melt(wide_df, id_vars=["data_object_id", "geo_loc_name"], var_name="taxa", value_name="relative_abundance")

    final_df = melted_df.groupby(["geo_loc_name", "taxa"])["relative_abundance"].mean().reset_index(name="avg_relative_abundance")

    return final_df

## Calculate the abundance of the location data frames

Using the function defined above, the `m_df` and the `o_df` data frames returned from iterating over the TSV files are used as input into the function, where the final abundance calculations and top 5 taxa are returned as data frames. We do the calculation by grouping by `geo_loc_name` in order to calculate abundances per location. We then concatenate the two data frames together, creating a new column for soil_horizon, where the value is either O or M, depending on which data frame it originally came from.

In [18]:
# caculate abundance for each soil horizon type and get top 5 taxa, grouping the rest
m_loc = loc_abund(m_df)
o_loc = loc_abund(o_df)

# combine data frames
o_loc["soil_horizon"] = "O"
m_loc["soil_horizon"] = "M"
loc_abund_df = pd.concat([o_loc, m_loc])

# Extract only region names from geo_loc_name
loc_abund_df["location"] = loc_abund_df["geo_loc_name"].str.extract(r'Colorado, (.*)')

loc_abund_df

,geo_loc_name,taxa,avg_relative_abundance,soil_horizon,location
0,"USA: Colorado, Niwot Ridge",Acidimicrobiia,0.431343,O,Niwot Ridge
1,"USA: Colorado, Niwot Ridge",Acidithiobacillia,0.029408,O,Niwot Ridge
2,"USA: Colorado, Niwot Ridge",Aconoidasida,0.011135,O,Niwot Ridge
3,"USA: Colorado, Niwot Ridge",Actinomycetes,22.746904,O,Niwot Ridge
4,"USA: Colorado, Niwot Ridge",Actinopteri,0.002615,O,Niwot Ridge
...,...,...,...,...,...
1143,"USA: Colorado, Rocky Mountains",unclassified Zoopagomycota,0.000000,M,Rocky Mountains
1144,"USA: Colorado, Rocky Mountains",unclassified candidate division NC10,0.095782,M,Rocky Mountains
1145,"USA: Colorado, Rocky Mountains",unclassified candidate division Zixibacteria,0.023120,M,Rocky Mountains
1146,"USA: Colorado, Rocky Mountains","unclassified dsDNA viruses, no RNA stage",0.000000,M,Rocky Mountains


## Plot the taxa abundance of M and O horizon soil samples for each location

Using the plotly library, the percent abundance of the taxa is plotted as a bar chart for each geo location and faceted by soil horizon.

In [19]:
geo_fig = px.bar(loc_abund_df, x = "soil_horizon", y="avg_relative_abundance", color = "taxa", 
                 facet_col = "location",
                 facet_col_spacing = 0.1,
                 title = "% Abundance of phylum-level taxa in M and O horizon samples for each Colorado location", 
                 labels = {"geo_loc_name": "Location", "avg_relative_abundance": "% Abundance"},
                 height = 600)
# update figure to remove "location=" from facet column labels
geo_fig.for_each_annotation(lambda a: a.update(text=a.text.replace("location=", "")))

# show figure
geo_fig.show()